# Web-scraping: сбор данных из баз данных и интернет-источников

## Управление браузером с Selenium: поиск элементов, запросы XPATH

*Алла Тамбовцева, НИУ ВШЭ*

### Часть 1: базовый поиск элементов на странице

Импортируем необходимые модули и компоненты:
    
* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запускаем браузер Chrome:

In [ ]:
br = wd.Chrome()

Продолжим работать с сайтом книжного магазина «Библио-Глобус». Отправляем запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [ ]:
br.get("https://www.biblio-globus.ru/")

Найдём на странице поле для ввода ключевых слов для поиска по его ID:

In [ ]:
search = br.find_element(By.ID, "SearchBooks")

Введём в это поле текст – запросим книги по Python:

In [ ]:
search.send_keys("Python")

Как мы уже знаем, для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

Используя метод `.send_keys()`, «отправьте» в поле `search` нажатие клавиши *Enter*. 

In [ ]:
### YOUR CODE HERE ###

Отлично! Теперь нужно поработать с результатами поиска. Технически, нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью BeautifulSoup. Но давайте мыслить более глобально – результаты не помещаются на одной странице, поэтому для общего решения нам надо научиться выяснять, сколько страниц пролистывать для сбора всех товаров по ключевым словам. 

Для этого нам нужно найти кнопку со стрелкой `»»` и внимательно её изучить. Давайте попробуем найти эту стрелку разными способами, чтобы познакомиться с разными атрибутами в `By`. 

Сначала найдём её последовательно по классам и тэгам (последовательно, потому что стрелка представлена ссылкой, а ссылок на странице много, нужно сначала зафиксировать более общий раздел, где её искать) и сохраним как `arrow`.

**Подсказка.** Сначала имеет смысл найти список со ссылками на разные страницы по классу.

In [ ]:
### YOUR CODE HERE ###

Посмотрим, что внутри. Можем запросить весь код HTML, который есть внутри найденного объекта (его тип *WebElement*):

In [ ]:
arrow.get_attribute('innerHTML')

В нашем случае полный код не очень интересен, потому что ссылка, которая отправляет нас на последнюю страницу с результатами, хранится не внутри тэгов, а в отдельном атрибуте `href` (как и обычно). Текст `»»` нам, скорее, полезен просто для того, чтобы убедиться, что мы нашли то, что нужно :) 

Запросим аналогичным образом `href`:

In [ ]:
arrow.get_attribute('href')

Вот эта информация уже гораздо полезнее, внутри этой ссылки есть указание на то, какая страница является последней. То есть, нам не придётся кликать на стрелки *Далее* до тех пор, пока мы не дойдём до конца, через какой-нибудь цикл while, мы сможем написать код, который будет пролистывать 10 страниц и забирать результаты. 

И да, эта ссылка, на которую мы вышли через Selenium, подсказывает нам, что и без Selenium можно справиться. Переход на каждую страницу сопровождается изменением ссылки, а значит, можно просто формировать запрос, подставляя в шаблон ссылки ключевые слова после `query` и номер страницы после `page`. Так бывает не всегда, иногда страница обновляется динамически, при активации каких-то процессов ссылка на страницу остаётся та же, а её содержимое меняется – в код HTML «встраиваются» новые объекты. Тогда без Selenium не обойтись, потому что без имитации конкретных действий необходимые объекты просто не появятся (страница или её часть просто не обновится, так как не запустится соответствующая функция на JavaScript, которая реагирует на действия пользователя).

**Вопрос.** Подумайте, как можно организовать код без использования Selenium, чтобы запрос пользователя отправлялся на сайт магазина и чтобы ему возвращались все результаты по запросу. 

Со страницами результатов разобрались, к ним мы вернёмся, чтобы забрать исходный код и ссылки на товары, а пока давайте посмотрим на ещё один способ поиска объектов – найдём нужную стрелку проще – по тексту ссылки.

In [ ]:
### YOUR CODE HERE ###

Использованные выше способы поиска хорошие, но не всегда надёжные и универсальные. В первом случае смущает поиск элемента внутри другого, да ещё и по индексу. Во втором случае потенциальных проблем меньше, но не всегда на нужной кнопке будет текст, там вообще может быть картинка. Поэтому давайте познакомимся с универсальным способом поиска через запросы XPATH.

### Часть 2: поиск с помощью запросов XPATH

Немного теории.

**XML** (от *eXtended MarkUp Language*) – язык разметки, только в отличие от HTML не позволяет регулировать внешний вид страницы, а просто хранит данные в виде строки с удобными тэгами.

На XML-файл можно смотреть как на хранилище, откуда по запросу динамически подгружаются данные для подстановки в HTML-файл. Смысл: когда нам нужно постоянно обновлять информацию на веб-странице (каталоги товаров в магазине, данные о погоде, курсе валют), не нужно каждый раз переписывать HTML-файл, достаточно изменить XML-файл, а из него уже информация «подтянется» на страницу с помощью запроса, написанного на JavaScript.

**XPATH** (от *XML Path Language*) – язык запросов в XML-файлу, который можно использовать и для HTML тоже.

Примеры запросов (честно взяты [отсюда](https://www.w3schools.com/xml/xpath_intro.asp), очень полезный тьюториал по XPATH, у них же есть классные материалы по XML):

* `//title[@lang]`: все элементы с тэгом `<title>`, имеющие атрибут `lang`;
* `//title[@lang='en']` : все элементы с тэгом `<title>`, имеющие атрибут `lang`, равный `'en'`;
* `//title[@*]`: все элементы с тэгом `<title>`, имеющие хоть какие-нибудь атрибуты.


Давайте воспользуемся поиском с помощью запроса XPATH и найдём стрелку-ссылку на последнюю страницу (внимание на атрибут `aria-label`).

In [ ]:
### YOUR CODE HERE ###

### Часть 3: собираем всё вместе и сохраняем результаты

Итак, допустим, у нас есть переменная `to_search` с ключевыми словами для поиска.  

In [ ]:
to_search = "Python"

Импортируем вспомогательные модули и функции:

In [ ]:
import re
from bs4 import BeautifulSoup
from time import sleep

Напишем код, который будет выполнять поиск по ключевым словам и сохранять ссылки на страницы товаров.

In [ ]:
br.get("https://www.biblio-globus.ru/")
sleep(1)

search = br.find_element(By.ID, "SearchBooks")
search.send_keys(to_search)
search.send_keys(Keys.ENTER)

arrow = br.find_element(By.XPATH, "//a[.//span[text()='»»']]")
link = arrow.get_attribute("href")

# выпендриваемся, вместо нескольких .split() 
# пользуемся регулярными выражениями

n = re.search('(page=)(\d+)', link).group(2)

In [ ]:
### YOUR CODE HERE ###

# для шаблона
# https://www.biblio-globus.ru/search?query=Python&page=9&sort=0&instock=&cat=0&isdiscount=